In [1]:
import pandas as pd
import csv
import re

# correzione dell'ordine di stampa
import functools
from operator import rshift
print = functools.partial(print, flush=True)

# import dei moduli per Content Analyzer, Recommender System e Evaluation come librerie
from clayrs import content_analyzer as ca
from clayrs import recsys as rs
from clayrs import evaluation as eva

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\glamo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# elaborazione del file .dat relativo agli items
path = 'C:/Users/glamo/Desktop/Repository/RecSys-Algorithms-Evaluation/Dataset/Movielens 1M/'

moviesCsv = open(path + 'movies.csv', 'w')
moviesCsv.write("id,title,year,genres\n")

with open(path + 'movies.dat', 'r') as moviesDat:
    for entry in moviesDat:
        entry = entry.split("::")
        moviesCsv.write(entry[0] + ',')
        year = re.findall('\d\d\d\d', entry[1])
        moviesCsv.write('"' + re.split(' \(\d\d\d\d\)', entry[1])[0] + '"' + ',')
        moviesCsv.write(year[0] + ',')
        entry[2] = re.sub('\|', ',', entry[2])
        entry[2] = re.sub('\n', '', entry[2])
        moviesCsv.write('"' + entry[2] + '"' + '\n')
        
moviesDat.close()

In [3]:
# elaborazione del file .dat relativo agli utenti
usersCsv = open(path + 'users.csv', 'w')
usersCsv.write('user_id,gender,age,occupation,zip_code\n')

with open(path + 'users.dat', 'r') as usersDat:
    for entry in usersDat:
        entry = entry.split("::")
        usersCsv.write(entry[0] + ',')
        usersCsv.write(entry[1] + ',')
        usersCsv.write(entry[2] + ',')
        usersCsv.write(entry[3] + ',')
        usersCsv.write(entry[4])
        
usersDat.close()

In [4]:
path = 'C:/Users/glamo/Desktop/Repository/RecSys-Algorithms-Evaluation/Dataset/'
# elaborazione del file .dat relativo ai ratings
ratingsCsv = open(path + 'ratings.csv', 'w')
ratingsCsv.write("user_id,movie_id,rating,timestamp\n")

with open(path + 'ratings.dat', 'r') as ratingsDat:
    for entry in ratingsDat:
        entry = entry.split("::")
        ratingsCsv.write(entry[0] + ',')
        ratingsCsv.write(entry[1] + ',')
        ratingsCsv.write(entry[2] + ',')
        ratingsCsv.write(entry[3])
        
ratingsDat.close()
ratingsCsv.close()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/glamo/Desktop/Repository/RecSys-Algorithms-Evaluation/Dataset/ratings.dat'

In [ ]:
# rappresentazione items con TF-IDF
movies_ca_config = ca.ItemAnalyzerConfig(
    source = ca.CSVFile(path + 'movies.csv'),
    id = 'id',
    output_directory = 'C:/Users/glamo/Desktop/Repository/RecSys-Algorithms-Evaluation/movies_codified/'
)

movies_ca_config.add_single_config(
    'title',
    ca.FieldConfig(ca.SkLearnTfIdf(),
    preprocessing = ca.NLTK(stopwords_removal=True, lemmatization=True),
    id = 'tfidf'              
    )
)

ca.ContentAnalyzer(config = movies_ca_config).fit()

In [ ]:
# rappresentazioni multiple
movies_multiple_config = ca.ItemAnalyzerConfig(
    source = ca.CSVFile(path + 'movies.csv'),
    id = 'id',
    output_directory = 'C:/Users/glamo/Desktop/Repository/RecSys-Algorithms-Evaluation/mult_movies_codified/'
)

movies_multiple_config.add_multiple_config(
    ( 
        'title',
        ca.FieldConfig(ca.SkLearnTfIdf(),
        preprocessing = ca.NLTK(stopwords_removal=True, lemmatization=True),
        id = 'tfidf'              
    ),
    (
        
    )
    
)

In [ ]:
# recommender system
ratings = ca.Ratings(ca.CSVFile(path + 'ratings.csv'))
print(ratings)

In [ ]:
centroid_vec = rs.CentroidVector(
    {'title': 'tfidf'},
    similarity = rs.CosineSimilarity()
)

In [ ]:
train_list, test_list = rs.KFoldPartitioning(n_splits=2).split_all(ratings)
first_train = train_list[0]

cbrs = rs.ContentBasedRS(centroid_vec, first_train, 'C:/Users/glamo/Desktop/Repository/RecSys-Algorithms-Evaluation/movies_codified/')

In [ ]:
first_test_set = test_list[0]

rank = cbrs.fit_rank(first_test_set, user_id_list=['8', '2', '1'], n_recs=3)

In [ ]:
print(rank)

In [ ]:
result_list = []

for train_set, test_set in zip(train_list, test_list):
    cbrs = rs.ContentBasedRS(centroid_vec, train_set, 'C:/Users/glamo/Desktop/Repository/RecSys-Algorithms-Evaluation/movies_codified/')
    rank_to_append = cbrs.fit_rank(test_set)
    result_list.append(rank_to_append)